In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv


In [2]:
df = pd.read_csv('/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv')

/tmp/ipykernel_55/4183513879.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv')


In [5]:
df = df.dropna()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712654 entries, 0 to 739175
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     712654 non-null  object
 1   title   712654 non-null  object
 2   text    712654 non-null  object
 3   topic   712654 non-null  object
 4   tags    712654 non-null  object
 5   date    712654 non-null  object
dtypes: object(6)
memory usage: 38.1+ MB


In [7]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [10]:
from sklearn.model_selection import train_test_split
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

df = df.drop(columns=['url', 'date', 'title'])

In [71]:
X = []
for tag in list(df.tags.unique()):
    if tag in ['Инновации', 'Вооружение', 'Мировой опыт']:
        continue
    X_tag = df[df['tags'] == tag].to_numpy()
    np.random.shuffle(X_tag)
    X_tag = X_tag[:100]
    X.extend(list(X_tag))
X = np.array(X)
np.random.shuffle(X)
X = pd.DataFrame(X)
X.columns = ['text', 'topic', 'tags']

In [74]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(X['tags'])
topic = X['topic']
X = X['text']

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)

In [85]:
import re
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)
    text = ' '.join(text.split())
    return text

def lemmatize(text):
    wnl = WordNetLemmatizer()
    tokens = word_tokenize(text)
    words = [wnl.lemmatize(t) for t in tokens]
    return ' '.join(words)

def remove_stopwords(text):
    stop_words = set(stopwords.words('russian'))
    tokens = word_tokenize(text)
    words = [t for t in tokens if t not in stop_words]
    return ' '.join(words)

def preprocess(text):
    text = clean(text)
    text = lemmatize(text)
    text = remove_stopwords(text)
    return text

X_train = X_train.apply(lambda x: preprocess(x))
X_test = X_test.apply(lambda x: preprocess(x))


In [94]:
tf = TfidfVectorizer(use_idf=False)
X_train_tf = tf.fit_transform(X_train)
X_test_tf = tf.transform(X_test)
tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [95]:
from sklearn.svm import LinearSVC


svc_tf = LinearSVC(C=0.1, class_weight='balanced')
svc_tfidf = LinearSVC(C=0.1, class_weight='balanced')
svc_tf.fit(X_train_tf, y_train)
svc_tfidf.fit(X_train_tfidf, y_train)

LinearSVC(C=0.1, class_weight='balanced')

In [97]:
from sklearn.metrics import f1_score

y_pred_tf = svc_tf.predict(X_test_tf)
y_pred_tfidf = svc_tfidf.predict(X_test_tfidf)
y_pred_tf_train = svc_tf.predict(X_train_tf)
y_pred_tfidf_train = svc_tfidf.predict(X_train_tfidf)
print(f'Train accuracy: \n TF: {f1_score(y_train, y_pred_tf_train, average='macro')} \n TF-IDF: {f1_score(y_train, y_pred_tfidf_train, average='macro')}')
print(f'Test accuracy: \n TF: {f1_score(y_test, y_pred_tf, average='macro')} \n TF-IDF: {f1_score(y_test, y_pred_tfidf, average='macro')}')

Train accuracy: 
 TF: 0.9583149787017607 
 TF-IDF: 0.997207382088445
Test accuracy: 
 TF: 0.5446804999474998 
 TF-IDF: 0.5586610481483871
